BERHASIL 1

In [113]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Average

# 1. Direktori dataset dan model
dataset_dir = r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Split_dataset"
test_dir = os.path.join(dataset_dir, "test")
model_paths = {
    "mobilenetv2": r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Model_Skripsi_NEWW (Layer)\mobilenetv2_Percobaan 4(3).h5",
    "xception": r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Model_Skripsi_NEWW (Layer)\Percobaan2(1)_Xception.h5",
    "efficientnetb0": r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\efficientnetb0oo_2.h5"
}

# 2. Label dataset
class_labels = ["Buras", "Dangkot", "Gogos", "Kapurung", "Sop Konro"]

# 3. Data generator untuk test
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),  # Ukuran input gambar (sesuaikan dengan model Anda)
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

# 4. Muat model dan kompilasi ulang jika diperlukan
models = []
for model_name, model_path in model_paths.items():
    model = load_model(model_path)
    # Kompilasi ulang model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    models.append(model)

# 5. Buat ensemble model tanpa training
predictions = [model.predict(test_generator) for model in models]

# 6. Ensemble prediksi dengan rata-rata
ensemble_preds = (predictions[0] + predictions[1] + predictions[2]) / 3

# 7. Ambil prediksi final sebagai kelas dengan probabilitas tertinggi
final_preds = np.argmax(ensemble_preds, axis=1)

# Ambil label sebenarnya
true_labels = test_generator.classes

# 8. Hitung metrik evaluasi
accuracy = accuracy_score(true_labels, final_preds)
precision = precision_score(true_labels, final_preds, average='weighted')
recall = recall_score(true_labels, final_preds, average='weighted')
f1 = f1_score(true_labels, final_preds, average='weighted')

# 9. Tampilkan hasil evaluasi
print(f"Akurasi  : {accuracy * 100:.2f}%")
print(f"Presisi  : {precision:.2f}")
print(f"Recall   : {recall:.2f}")
print(f"F1-Score : {f1:.2f}")

# 10. Simpan ensemble model jika diperlukan
ensemble_model = models[0]  # Bisa menggunakan salah satu model yang ada
ensemble_model.save("ensemble_model.h5")
print("Ensemble model saved as ensemble_model.h5")


Found 1020 images belonging to 6 classes.


d:\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


32/32 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 144s 4s/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 102s 2s/step


Akurasi  : 90.88%
Presisi  : 0.91
Recall   : 0.91
F1-Score : 0.91
Ensemble model saved as ensemble_model.h5


In [105]:
from tensorflow.keras.applications import MobileNetV2, Xception, EfficientNetB0
from tensorflow.keras.layers import Average, Input, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import clone_model
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

dic = {0: 'Buras', 1: 'Dangkot', 2: 'Gogos', 3: 'Kapurung', 4: 'Sokko', 5: 'Sop Konro'}

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Load the pre-trained models
mobilenetv2_model = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Model_Skripsi_NEWW (Layer)\mobilenetv2_Percobaan 4(3).h5", compile=False)
xception_model = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Model_Skripsi_NEWW (Layer)\Percobaan2(1)_Xception.h5",compile=False)
efficientnetb0_model = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\efficientnetb0oo_2.h5",compile=False)

print(mobilenetv2_model.name)  
print(xception_model.name)   
print(efficientnetb0_model.name) 

# print(mobilenetv2_model.summary())
# print(xception_model.summary())
# print(efficientnetb0_model.summary())


MobileNetV2
Xception
efficientnetb0


In [107]:
input_layer = Input(shape=(224, 224, 3))

In [108]:
mobilenetv2_out = mobilenetv2_model(input_layer)
xception_out = xception_model(input_layer)
efficientnetb0_out = efficientnetb0_model(input_layer)

In [109]:
concat_out = Concatenate()([mobilenetv2_out, xception_out, efficientnetb0_out])
#concat_out = Concatenate()([mobilenetv2_out, xception_out])

In [110]:
# Create the ensemble model
ensemble_model = Model(inputs=input_layer, outputs=concat_out, name='ensemble_model')

In [111]:
from keras.models import load_model

try:
    model = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\efficientnetb0oo_2.h5")
    print("Model loaded successfully.")
except ValueError as e:
    print(f"Error loading model: {e}")

import h5py

file_path = r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\efficientnetb0oo_2.h5"

with h5py.File(file_path, 'r') as h5_file:
    print("Keys in .h5 file:", list(h5_file.keys()))



Model loaded successfully.
Keys in .h5 file: ['model_weights', 'optimizer_weights']


part 2 di atas

In [112]:
ensemble_model.save("ensemble_model2.h5")
print("Ensemble model successfully saved in SavedModel format.")

TypeError: cannot pickle 'module' object

In [101]:
# Define a new input layer
input_layer = Input(shape=(224, 224, 3))

model = [mobilenetv2_model,
        xception_model,
        efficientnetb0_model
]

# mobilenetv2_out = mobilenetv2_model(input_layer)
# xception_out = xception_model(input_layer)
# efficientnetb0_out = efficientnetb0_model(input_layer)

# # Combine the outputs using Concatenate layer
# concat_out = Concatenate()([mobilenetv2_out, xception_out, efficientnetb0_out])

# # Create the ensemble model
# ensemble_model = Model(inputs=input_layer, outputs=concat_out)

# # model_input = Input(shape=(224, 224, 3))

model_outputs = [model(input_layer)[0] for model in models]

ensemble_output = Average()(model_outputs)

ensemble_model = Model(inputs=input_layer, outputs=ensemble_output, name='ensemble_cnn')

# Save the ensemble model
ensemble_model.save("ensemble_modelll.h5")
print("Ensemble model successfully saved as 'ensemble_model1.h5'.")

# Function for ensemble prediction
def ensemble_predict(img_path):
    img = preprocess_image(img_path)
    # Predict using all models
    predictions = [model.predict(img) for model in [mobilenetv2_model, xception_model, efficientnetb0_model]]
    average_prediction = np.mean(predictions, axis=0)
    return np.argmax(average_prediction, axis=-1)

# Evaluating the model
def evaluate_model(test_images, test_labels):
    true_labels = []
    pred_labels = []
    
    for img_path, true_label in zip(test_images, test_labels):
        pred_label = ensemble_predict(img_path)[0]
        true_labels.append(true_label)
        pred_labels.append(pred_label)
    
    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, average='weighted')
    recall = recall_score(true_labels, pred_labels, average='weighted')
    f1 = f1_score(true_labels, pred_labels, average='weighted')
    
    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1)
    print("Classification Report:\n", classification_report(true_labels, pred_labels, target_names=list(dic.values())))

# Example test images and labels
test_images = [
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Buras\BURAS_3.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Dangkot\DANGKOT_7.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Gogos\GOGOS_22.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Kapurung\KAPURUNG_27.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Sokko\SOKKO_6.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Sop Konro\SOP KONRO_188.jpg"
]
test_labels = [0, 1, 2, 3, 4, 5]

# Evaluate the ensemble model
evaluate_model(test_images, test_labels)


ValueError: The name "functional_1" is used 3 times in the model. All operation names should be unique.